In [7]:
%pip install --quiet \ pandas 

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [ ]:
dados = [
    ("associação de Agricultores Familiares da Eco Comunidade do Assentamento 15 de Agosto (Afeca)", ["São Sebastião"], -15.91117477568832, -47.72142914924957),
    ("Cooperativa Agrícola Buriti Vermelho (Cooper-Horti) - Paranoá", ["Paranoá"], -15.7485, -47.6465),
    ("Cooperativa dos Produtores Rurais de Planaltina de Goiás e Região (Prorural)", ["Paranoá", "Plano Piloto"], -15.445611703837635, -47.6190618576721),
    ("Cooperativa Agropecuária da Região de Brazlândia (Coopebraz)", ["Brazlândia", "Taguatinga", "Samambaia", "Recanto das Emas"], -15.60313666754234, -48.11359575477609),
    ("Cooperativa de Agricultura Familiar Mista do Distrito Federal (Cooper-mista)", ["Planaltina"], -15.764378385278865, -47.4937971768077),
    ("Cooperativa Agrícola da Região de Planaltina (Cootaquara) ",["Ceilândia"],-15.633189282812694, -47.5225251020717),
]


